In [22]:
import insightface
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics.pairwise import cosine_similarity
from scipy import linalg
import math
import os

In [13]:
cosine_threshold = 0.4
color = (0, 0, 255)
ctx_id = 0
font = cv2.FONT_HERSHEY_SIMPLEX

interest_zone_bbox = [(681, 225), (1318, 823)]

In [23]:
cap = cv2.VideoCapture('videos/video1.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

!rm -f head_pose_tracking.avi
!rm -f head_pose_tracking.mp4
result = cv2.VideoWriter('head_pose_tracking.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

In [15]:
model = insightface.app.FaceAnalysis()
model.prepare(ctx_id = ctx_id, nms=0.4)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [16]:
def is_rotation_matrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rotation_vector_to_euler_angles(rotation_vector) :
    R, _ = cv2.Rodrigues(rotation_vector)
    
    if not is_rotation_matrix(R):
        return
    
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.rad2deg(np.array([x, y, z]))

In [17]:
def chin_calc(bbox, fivepointlandmarks):
    p1 = np.asarray([bbox[2], bbox[3]])
    p2 = np.asarray([bbox[0], bbox[3]])
    p3 = np.asarray([int((fivepointlandmarks[6] + fivepointlandmarks[8]) / 2), int((fivepointlandmarks[7] + fivepointlandmarks[9]) / 2)])
    chindistfromp3 = int(linalg.norm(np.cross(p2 - p1, p1 - p3)) / linalg.norm(p2 - p1))
    chin = [p3[0], p3[1] + chindistfromp3]

    # Return a 2-D point represent for the chin of the given face
    return chin

def six_point_of_landmarks(bbox, fivepointlandmarks):
    pchin = chin_calc(bbox, fivepointlandmarks)

    image_points = np.array([
                            (fivepointlandmarks[4], fivepointlandmarks[5]),     # Nose tip
                            (pchin[0], pchin[1])                          ,     # Chin
                            (fivepointlandmarks[0], fivepointlandmarks[1]),     # Left eye left corner
                            (fivepointlandmarks[2], fivepointlandmarks[3]),     # Right eye right corne
                            (fivepointlandmarks[6], fivepointlandmarks[7]),     # Left Mouth corner
                            (fivepointlandmarks[8], fivepointlandmarks[9])      # Right mouth corner
                        ], dtype="double")
    return image_points

def est_head_pose(face, imsize):
    bbox = face.bbox.astype(np.int).flatten()
    fivepointlandmarks = face.landmark.astype(np.int).flatten()
    image_points = six_point_of_landmarks(bbox, fivepointlandmarks)

    # 3D model points. 
    model_points = np.array([
                                (0.0, 0.0, 0.0),             # Nose tip
                                (0.0, -330.0, -65.0),        # Chin
                                (-210.0, 170.0, -135.0),     # Left eye left corner
                                (210.0, 170.0, -135.0),      # Right eye right corne
                                (-150.0, -150.0, -125.0),    # Left Mouth corner
                                (150.0, -150.0, -125.0)      # Right mouth corner
                            ])
    
    # Camera internals
    focal_length = imsize[1]
    center = (imsize[1]/2, imsize[0]/2)
    camera_matrix = np.array(
                            [[focal_length, 0, center[0]],
                            [0, focal_length, center[1]],
                            [0, 0, 1]], dtype = "double"
                            )
    
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, 
                                                                  image_points, 
                                                                  camera_matrix, 
                                                                  dist_coeffs, 
                                                                  flags=cv2.cv2.SOLVEPNP_ITERATIVE)
    (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), 
                                                                rotation_vector, 
                                                                translation_vector, 
                                                                camera_matrix, 
                                                                dist_coeffs)

    p1 = (int(image_points[0][0]), int(image_points[0][1]))
    p2 = (int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
    
    euler_angles = rotation_vector_to_euler_angles(rotation_vector)

    # return a line that point out the current pose of the head
    return p1, p2, bbox, euler_angles

In [18]:
def face_filter(raw_euler_angles):
    roll = False
    pitch = False
    yaw = False
    
    euler_angles = np.absolute(raw_euler_angles.astype(int))
    
    if (170 <= euler_angles[0] and euler_angles[0] <= 180):
        roll = True
    if (170 <= euler_angles[2] and euler_angles[2] <= 180):
        yaw = True
    if 35 <= euler_angles[1] and euler_angles[1] <= 50:
        pitch = True
        
    return roll and pitch and yaw

In [19]:
def check_point_in_rect(rect, points):
    # rect[0] = top left point of the rectangle, rect[1] = bottom right of the rectangle 
    top_left, bottom_right = rect[0], rect[1]
    
    for point in points:
        x_bound = top_left[0] <= point[0] and bottom_right[0] >= point[0]
        y_bound = top_left[1] <= point[1] and bottom_right[1] >= point[1]
        if not (x_bound and y_bound):
            return False
    return True

In [24]:
face_list = []
dist_threshold = .5
def face_tagger(face_vector):
    if not face_list:
        face_list.append(face_vector)
        return 0
    
    max_index = -1
    max_dist = -1
    for i in range(len(face_list)):
        current_dist = cosine_similarity([face_list[i]], [face_vector])[0]
        if current_dist > max_dist:
            max_dist = current_dist
            max_index = i
            
    if max_dist >= dist_threshold:
        return max_index
    else:
        face_list.append(face_vector)
        return len(face_list)

In [25]:
frameNo = 0
while(True):
    print(frameNo)
    ret, frame = cap.read()
    cv2.rectangle(frame, (interest_zone_bbox[0][0], interest_zone_bbox[0][1]), (interest_zone_bbox[1][0], interest_zone_bbox[1][1]), (255, 0, 0), 3)
    
    if ret is True:
        faces = model.get(frame)
        for idx, face in enumerate(faces):
            bbox = face.bbox.astype(np.int).flatten()
            if not check_point_in_rect(interest_zone_bbox, [(bbox[0], bbox[1]), (bbox[2], bbox[3])]):
                continue
            p1, p2, bbox, euler_angles = est_head_pose(face, frame.shape)
            
            if face_filter(euler_angles):
                cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 3)
                embedding = face.embedding.astype(np.float).flatten()
                index = face_tagger(embedding)
                cv2.putText(frame, str(index), (bbox[0] - 20, bbox[1] - 20), font, 2, (255, 0, 0), 2, cv2.LINE_AA)
            # else:
                # cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 3)
            # cv2.line(frame, p1, p2, (255,0,0), 2)
            # cv2.putText(frame, str(np.absolute(euler_angles.astype(int))), (bbox[0] - 20, bbox[1] - 20), font, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        result.write(frame)
        print('Face detected: ' + str(len(face_list)))
        frameNo += 1
        clear_output(wait=True)
cap.release()
result.release()
print('Render completed!')

369


ValueError: Expected 2D array, got 1D array instead:
array=[-3.26959223e-01  6.87067747e-01  3.94089550e-01  5.07947132e-02
 -8.35412621e-01  1.02034599e-01  3.81974697e-01 -7.08948225e-02
  8.69914949e-01  2.36705437e-01 -3.38939130e-01 -5.28084457e-01
 -4.79741991e-01  1.78914309e-01  1.04542829e-01 -7.23256394e-02
 -5.93884587e-01  4.03793544e-01 -3.43498170e-01 -1.59600854e-01
 -3.20078939e-01  1.83209285e-01 -1.98669076e-01  1.06141180e-01
 -1.97738469e-01  4.51026201e-01 -4.59874988e-01 -8.36340934e-02
  5.84345162e-01 -2.97379166e-01  2.01182544e-01 -6.55476391e-01
 -4.59255069e-01  2.14174941e-01  1.56605095e-01  1.00872684e+00
 -3.85751247e-01  5.80046594e-01  1.39400721e-01 -2.27422282e-01
 -5.97222388e-01 -9.63216126e-02 -2.49296844e-01  1.86820582e-01
  3.20102215e-01  1.04161784e-01 -2.70198405e-01  2.91778654e-01
  5.56108132e-02 -6.93405345e-02 -3.39203745e-01 -7.37273574e-01
  6.25839755e-02  5.24970114e-01 -4.07205880e-01  1.44179896e-01
 -3.68606597e-01  3.49231333e-0